# Running models and cross validation 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn import metrics

In [3]:
#Time series split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

In [5]:
# Loading final dataset
df = pd.read_csv('all_features_merged.csv')
print(df.head())
print(df.shape)

#Counting NA
#nan_count = df['water_reservoirs_and_hydro_storage'].isna().sum()
#print(nan_count)

                        Date Country  Unnamed: 0  Solar  Wind Onshore  \
0        2014-12-31 23:00:00      PT         NaN    NaN           NaN   
1  2014-12-31 23:00:00+00:00      GB         0.0    0.0        4546.0   
2  2014-12-31 23:30:00+00:00      GB         1.0    0.0        4546.0   
3        2015-01-01 00:00:00      BE         NaN    NaN           NaN   
4        2015-01-01 00:00:00      CH         NaN    NaN           NaN   

   Wind Offshore  index  water_reservoirs_and_hydro_storage  energy_price  \
0            NaN    0.0                                 NaN           NaN   
1         3165.0    NaN                                 NaN           NaN   
2         3165.0    NaN                                 NaN           NaN   
3            NaN    1.0                                 NaN           NaN   
4            NaN    2.0                                 NaN         44.94   

   biomass     gas  nuclear  
0    310.0   575.0      NaN  
1      NaN     NaN      NaN  
2      N

In [7]:
# Separating features and target variable
y = df['energy_price']
X = df.drop(['energy_price'], axis=1)
y.head()

0      NaN
1      NaN
2      NaN
3      NaN
4    44.94
Name: energy_price, dtype: float64

In [8]:
#Train-test split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting the data
split_point = int(len(y) * 0.8)
y_train, y_test = y[0:split_point], y[split_point:]
X_train, X_test = X[0:split_point], X[split_point:]
df_train, df_test = df[0:split_point], df[split_point:]
print(y_train)

0             NaN
1             NaN
2             NaN
3             NaN
4           44.94
            ...  
5103713     49.95
5103714    320.00
5103715    309.64
5103716    375.61
5103717    309.64
Name: energy_price, Length: 5103718, dtype: float64


In [12]:
# Fit model
model = ARIMA(y_train, order=(1,0,0))  # example order, adjust based on your data
fitted_model = model.fit()


# Forecast
predictions = fitted_model.forecast(steps=len(y_test))



In [13]:
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'MSE: {mse}')

# Plot
plt.figure(figsize=(10, 6))
plt.plot(test.index, test, label='Actual')
plt.plot(test.index, predictions, label='Forecasted', color='red')
plt.legend()
plt.show()

print(predictions)


ValueError: Input contains NaN.

In [10]:
print(fitted_model)